@Copyright IQIYI 2021
http://challenge.ai.iqiyi.com/

In [1]:
#根据此题需求，最终训练的数据一个用户只有一行样本，所以在构建特征时会根据用户做很多统计特征

In [2]:
import pandas as pd
import numpy as np
from itertools import groupby
import gc

In [3]:
input_dir = "./data/raw_data/"
output_dir = "./data/new_data_B/"

In [4]:
launch = pd.read_csv(input_dir + "app_launch_logs.csv")
launch.date.min(), launch.date.max(),launch.head()

(100,
 222,
     user_id  launch_type  date
 0  10157996            0   129
 1  10139583            0   129
 2  10277501            0   129
 3  10099847            0   129
 4  10532773            0   129)

In [5]:
launch.sort_values('date',inplace=True)    #按时间排序，然后后续分组后用户的序列能够按时间排序

In [6]:
launch.head()

,user_id,launch_type,date
1462388,10123423,1,100
5290981,10505032,0,100
5290982,10504229,0,100
5290983,10049901,0,100
5290984,10324490,0,100


In [7]:
#构建序列
launch_grp = pd.DataFrame()

user_id = []
launch_date_str = []
launch_type_str = []
for i in launch.groupby('user_id'):
    launch_date = []
    launch_type = []
    user_id.append(i[0])
    for j in i[1]['date']:
        launch_date.append(j)
    for j in i[1]['launch_type']:
        launch_type.append(j)
        
    launch_date_str.append(str(launch_date))
    launch_type_str.append(str(launch_type))
launch_grp['user_id'] = list(user_id)
launch_grp['launch_date'] = list(launch_date_str)
launch_grp['launch_type'] = list(launch_type_str)
launch_grp.head()

,user_id,launch_date,launch_type
0,10000000,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[114, 117, 118]","[0, 0, 0]"
2,10000002,"[128, 129, 140, 144, 160, 163]","[0, 0, 0, 0, 0, 0]"
3,10000003,"[144, 144]","[1, 0]"
4,10000004,"[154, 155, 156, 157]","[0, 0, 0, 0]"


In [8]:
# launch_grp = launch.groupby("user_id").agg(
#     launch_date=("date", list),
#     launch_type=("launch_type", list)
# ).reset_index()
# launch_grp

In [9]:
launch_grp['launch_date'] = launch_grp['launch_date'].apply(lambda x: eval(x))
launch_grp['launch_type'] = launch_grp['launch_type'].apply(lambda x: eval(x))
launch_grp.head()

,user_id,launch_date,launch_type
0,10000000,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[114, 117, 118]","[0, 0, 0]"
2,10000002,"[128, 129, 140, 144, 160, 163]","[0, 0, 0, 0, 0, 0]"
3,10000003,"[144, 144]","[1, 0]"
4,10000004,"[154, 155, 156, 157]","[0, 0, 0, 0]"


In [10]:
# generating a sample for each user, a sample should has an anchor date for label
def choose_end_date(launch_date):
    n1, n2 = min(launch_date), max(launch_date)
    if n1 < n2 - 7:
        end_date = np.random.randint(n1, n2 - 7)  
        #end_date = n2 - 7                               #这里为了让训练集的序列长一点，固定一个值
    else:
        end_date = np.random.randint(130, 222 - 7)

    return end_date
launch_grp["end_date"] = launch_grp.launch_date.apply(choose_end_date)
launch_grp

,user_id,launch_date,launch_type,end_date
0,10000000,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",211
1,10000001,"[114, 117, 118]","[0, 0, 0]",173
2,10000002,"[128, 129, 140, 144, 160, 163]","[0, 0, 0, 0, 0, 0]",128
3,10000003,"[144, 144]","[1, 0]",199
4,10000004,"[154, 155, 156, 157]","[0, 0, 0, 0]",167
...,...,...,...,...
599995,10599995,"[125, 155, 155, 165, 174, 175, 186, 189, 190, ...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",180
599996,10599996,"[184, 185, 187, 189, 199, 206, 207, 209, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",198
599997,10599997,"[110, 115, 134, 157, 158, 159, 160, 161, 163, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",119
599998,10599998,"[107, 137, 159, 160, 182]","[0, 0, 0, 0, 0]",171


In [11]:
def get_label(row):
    launch_list = row.launch_date
    end = row.end_date
    label = sum([1 for x in set(launch_list) if end < x < end+8])
    return label

def get_train_launch_date(row):
    count = 0
    launch_date_list = row.launch_date
    for i in launch_date_list:
        if row.end_date>=i:
            count += 1
        else:
            break
    
    return launch_date_list[:count]
    
def get_train_launch_type(row):
    count = 0
    launch_type_list = row.launch_type
    launch_date_list = row.launch_date
    for i in launch_date_list:
        if row.end_date>=i:
            count += 1
        else:
            break
    return launch_type_list[:count]

launch_grp["label"] = launch_grp.apply(get_label, axis=1)
launch_grp["new_launch_date"] = launch_grp.apply(get_train_launch_date,axis=1)
launch_grp["new_launch_type"] = launch_grp.apply(get_train_launch_type,axis=1)


In [12]:
#有空值
# launch_grp.new_launch_date.fillna('[]',inplace=True)
# launch_grp.new_launch_type.fillna('[]',inplace=True)

In [13]:
launch_grp.head()

,user_id,launch_date,launch_type,end_date,label,new_launch_date,new_launch_type
0,10000000,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,"[114, 117, 118]","[0, 0, 0]",173,0,"[114, 117, 118]","[0, 0, 0]"
2,10000002,"[128, 129, 140, 144, 160, 163]","[0, 0, 0, 0, 0, 0]",128,1,[128],[0]
3,10000003,"[144, 144]","[1, 0]",199,0,"[144, 144]","[1, 0]"
4,10000004,"[154, 155, 156, 157]","[0, 0, 0, 0]",167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]"


In [14]:
# def get_label(row):
#     launch_list = row.launch_date
#     end = row.end_date
#     label = sum([1 for x in set(launch_list) if end < x < end+8])
#     return label
# launch_grp["label"] = launch_grp.apply(get_label, axis=1)
# launch_grp.head()

In [15]:
train = launch_grp[["user_id", "end_date", "label"]]
train.head()

,user_id,end_date,label
0,10000000,211,2
1,10000001,173,0
2,10000002,128,1
3,10000003,199,0
4,10000004,167,0


In [16]:
# reading test data
test = pd.read_csv(input_dir + "test_B.csv")
test["label"] = -1
test.head()

,user_id,end_date,label
0,10354569,187,-1
1,10594476,175,-1
2,10032866,177,-1
3,10467002,185,-1
4,10520906,208,-1


In [17]:
# concat train and test data
data = pd.concat([train, test], ignore_index=True)
data.head()

,user_id,end_date,label
0,10000000,211,2
1,10000001,173,0
2,10000002,128,1
3,10000003,199,0
4,10000004,167,0


In [18]:
len(data)

635000

# launch data process

In [19]:
tmp = test.merge(launch_grp[["user_id", "launch_type", "launch_date",]], how="left", on="user_id")
tmp.head()

,user_id,end_date,label,launch_type,launch_date
0,10354569,187,-1,"[0, 0, 0, 0, 0, 0, 0, 0]","[121, 122, 136, 138, 141, 143, 157, 167]"
1,10594476,175,-1,"[0, 0, 1, 0, 0, 0, 0]","[108, 111, 157, 157, 168, 169, 170]"
2,10032866,177,-1,"[0, 0]","[103, 122]"
3,10467002,185,-1,"[0, 0, 0, 0, 0, 0]","[122, 129, 136, 141, 142, 143]"
4,10520906,208,-1,"[0, 0, 0, 0, 0, 0, 0, 0]","[134, 159, 162, 165, 181, 187, 192, 202]"


In [20]:
#查看测试集中是否存在end_date是在训练集之前的
# counts = 0
# userid = []
# for i in range(len(tmp)):
#     max_date = max(tmp.loc[i,'launch_date'])
#     min_date = min(tmp.loc[i,'launch_date'])
#     if tmp.loc[i,'end_date'] >=min_date and tmp.loc[i,'end_date']<=max_date:
#         userid.append(tmp.loc[i,'user_id'])
        #print(tmp.iloc[i].values)
#print(counts)#有3941个测试集样本的最终时间是训练集的launch_date中的最后一天，

In [21]:
#将end_date拼到launchdate上


In [22]:
#此时才能删除原始的launch，因为上面已经和test合并merge了，
launch_grp = launch_grp.drop(['launch_type','launch_date'],axis=1)
launch_grp = launch_grp.rename(columns={'new_launch_type':'launch_type','new_launch_date':'launch_date'})

In [23]:
launch_grp = pd.concat([launch_grp,tmp],axis=0)   #此时test中的launch是比train中的长的，是包含关系
launch_grp

,user_id,end_date,label,launch_date,launch_type
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]"
2,10000002,128,1,[128],[0]
3,10000003,199,0,"[144, 144]","[1, 0]"
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]"
...,...,...,...,...,...
34995,10517398,216,-1,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
34996,10110309,172,-1,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]"
34997,10528272,220,-1,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
34998,10395221,195,-1,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]"


In [24]:
launch_grp = launch_grp.reset_index(drop=True)

In [25]:
del tmp
gc.collect()

40

In [26]:
launch_grp

,user_id,end_date,label,launch_date,launch_type
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]"
2,10000002,128,1,[128],[0]
3,10000003,199,0,"[144, 144]","[1, 0]"
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]"
...,...,...,...,...,...
634995,10517398,216,-1,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
634996,10110309,172,-1,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]"
634997,10528272,220,-1,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
634998,10395221,195,-1,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]"


In [29]:
#构建登录的统计特征,注意只用结束时间以前的序列来构建特征，否则会穿越。上述已经解决了穿越问题
launch_grp['launch_times'] = [len(v) for v in launch_grp.launch_date.values]
launch_grp['launch_type_0'] = [len(v)-sum(v) for v in launch_grp.launch_type.values]
launch_grp['launch_type_1'] = [sum(v) for v in launch_grp.launch_type.values]
launch_grp['launch_type_01rate'] = [sum(v)/len(v) if len(v)>0 else 0 for v in launch_grp.launch_type.values]
launch_grp['start_end_launch'] = [max(v)-min(v) if len(v)>0 else 0 for v in launch_grp.launch_date.values]

#计算launch_date的序列长度
launch_date_len = []
for i in launch_grp.launch_date:
    launch_date_len.append(len(i))
launch_grp['launch_date_len'] = launch_date_len

launch_grp.head()

,user_id,end_date,label,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_date_len
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.0,71,16
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.0,4,3
2,10000002,128,1,[128],[0],1,1,0,0.0,0,1
3,10000003,199,0,"[144, 144]","[1, 0]",2,1,1,0.5,0,2
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.0,3,4


In [30]:
# get latest 32 days([end_date-31, end_date]) launch type sequence
# 0 for not launch, 1 for launch_type=0, and 2 for launch_type=1
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-31, end+1)]
    return seq
launch_grp["launch_seq_31"] = launch_grp.apply(gen_launch_seq, axis=1)

In [31]:
# get latest 16 days([end_date-15, end_date]) launch type sequence
# 0 for not launch, 1 for launch_type=0, and 2 for launch_type=1
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-15, end+1)]
    return seq
launch_grp["launch_seq_15"] = launch_grp.apply(gen_launch_seq, axis=1)
launch_grp

,user_id,end_date,label,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_date_len,launch_seq_31,launch_seq_15
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,71,16,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,[128],[0],1,1,0,0.000000,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,10000003,199,0,"[144, 144]","[1, 0]",2,1,1,0.500000,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634995,10517398,216,-1,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,83,36,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]"
634996,10110309,172,-1,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,53,6,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634997,10528272,220,-1,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,57,22,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634998,10395221,195,-1,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,44,5,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [32]:
# get latest 8 days([end_date-7, end_date]) launch type sequence
# 0 for not launch, 1 for launch_type=0, and 2 for launch_type=1
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-7, end+1)]
    return seq
launch_grp["launch_seq_7"] = launch_grp.apply(gen_launch_seq, axis=1)
launch_grp

,user_id,end_date,label,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_date_len,launch_seq_31,launch_seq_15,launch_seq_7
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,71,16,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,[128],[0],1,1,0,0.000000,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 1]"
3,10000003,199,0,"[144, 144]","[1, 0]",2,1,1,0.500000,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634995,10517398,216,-1,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,83,36,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]","[1, 0, 1, 0, 0, 1, 1, 0]"
634996,10110309,172,-1,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,53,6,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
634997,10528272,220,-1,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,57,22,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
634998,10395221,195,-1,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,44,5,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"


In [33]:
#构造最近30/15/7天的登录次数特征
launch_grp['launch_times_31'] = launch_grp['launch_seq_31'].apply(lambda x:sum([1 for i in x if i!=0]))
launch_grp['launch_times_15'] = launch_grp['launch_seq_15'].apply(lambda x:sum([1 for i in x if i!=0]))
launch_grp['launch_times_7'] = launch_grp['launch_seq_7'].apply(lambda x:sum([1 for i in x if i!=0]))
launch_grp.head()

,user_id,end_date,label,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_date_len,launch_seq_31,launch_seq_15,launch_seq_7,launch_times_31,launch_times_15,launch_times_7
0,10000000,211,2,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.0,71,16,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",11,4,0
1,10000001,173,0,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.0,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
2,10000002,128,1,[128],[0],1,1,0,0.0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 1]",1,1,1
3,10000003,199,0,"[144, 144]","[1, 0]",2,1,1,0.5,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
4,10000004,167,0,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.0,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",4,4,0


In [34]:
launch_grp.columns

Index(['user_id', 'end_date', 'label', 'launch_date', 'launch_type',
       'launch_times', 'launch_type_0', 'launch_type_1', 'launch_type_01rate',
       'start_end_launch', 'launch_date_len', 'launch_seq_31', 'launch_seq_15',
       'launch_seq_7', 'launch_times_31', 'launch_times_15', 'launch_times_7'],
      dtype='object')

In [35]:
data = data.merge(
    launch_grp[['end_date', 'label', 'launch_date', 'launch_type', 'user_id',
       'launch_times', 'launch_type_0', 'launch_type_1', 'launch_type_01rate',
       'start_end_launch', 'launch_seq_31', 'launch_seq_15', 'launch_seq_7',
       'launch_times_31', 'launch_times_15', 'launch_times_7']],
    on=["user_id", "end_date", "label"],
    how="left"
)
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_seq_31,launch_seq_15,launch_seq_7,launch_times_31,launch_times_15,launch_times_7
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,71,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",11,4,0
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
2,10000002,128,1,1,[128],[0],1,1,0,0.000000,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 1]",1,1,1
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.500000,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",4,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634995,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,83,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]","[1, 0, 1, 0, 0, 1, 1, 0]",11,7,4
634996,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,53,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,0,0
634997,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,57,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",1,0,0
634998,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,44,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",1,0,0


In [36]:
#删除不需要的内存数据
del launch_grp
gc.collect()

80

# playback data and video data process

In [37]:
# 这个数据集很占内存，
# choose playback data in [end_date-31, end_date]
playback = pd.read_csv(input_dir + "user_playback_data.csv", dtype={"item_id": str})
playback = playback.merge(data, how="inner", on="user_id")
playback = playback.loc[(playback.date >= playback.end_date-31) & (playback.date <= playback.end_date)]
playback

,user_id,item_id,playtime,date,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,start_end_launch,launch_seq_31,launch_seq_15,launch_seq_7,launch_times_31,launch_times_15,launch_times_7
0,10057286,20628283,2208.612,145,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,4,0,0.0,30,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0
1,10057286,20628283,332.351,150,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,4,0,0.0,30,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0
5,10057286,24986210,74.370,161,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,4,0,0.0,30,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0
6,10057286,23148200,110.134,145,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,4,0,0.0,30,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0
8,10057286,23821440,1458.661,161,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,4,0,0.0,30,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77135345,10450556,23026777,3209.858,144,163,0,2,"[142, 144]","[0, 0]",2,2,0,0.0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",2,0,0
77135346,10073907,20928772,1.634,144,165,1,2,"[144, 151]","[1, 0]",2,1,1,0.5,7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",2,1,0
77135348,10508542,20933156,4.070,144,173,0,2,"[144, 144]","[0, 1]",2,1,1,0.5,0,"[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",1,0,0
77135351,10380139,23439637,23.733,144,150,0,2,"[144, 144]","[0, 1]",2,1,1,0.5,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]","[0, 2, 0, 0, 0, 0, 0, 0]",1,1,1


In [38]:
#最近31天播放时长
playtime = playback.groupby('user_id',as_index=False)['playtime'].sum()
playtime = playtime.rename(columns={'playtime':'playtime_31'})

In [39]:
#最近15天播放时长
playback_15 = playback.loc[(playback.date >= playback.end_date-15) & (playback.date <= playback.end_date)]
playtime_15 = playback_15.groupby('user_id',as_index=False)['playtime'].sum()
playtime_15 = playtime_15.rename(columns={'playtime':'playtime_15'})
playtime = playtime.merge(playtime_15,on='user_id',how='left')

In [40]:
#最近7天播放时长
playback_7 = playback.loc[(playback.date >= playback.end_date-7) & (playback.date <= playback.end_date)]
playtime_7 = playback_7.groupby('user_id',as_index=False)['playtime'].sum()
playtime_7 = playtime_7.rename(columns={'playtime':'playtime_7'})
playtime = playtime.merge(playtime_7,on='user_id',how='left')
playtime = playtime.fillna(0)
playtime.head()

,user_id,playtime_31,playtime_15,playtime_7
0,10000000,2587.170,0.000,0.000
1,10000002,19.925,19.925,19.925
2,10000004,5248.377,5248.377,0.000
3,10000005,1475.583,1475.583,1475.583
4,10000007,5782.044,5782.044,5782.044


In [41]:
len(playtime)

418287

In [42]:
data = data.merge(playtime,on='user_id',how='left')

In [43]:
del playtime
gc.collect()

20

In [44]:
# add video info to playback data
video_data = pd.read_csv(input_dir + "video_related_data.csv", dtype=str)
playback = playback.merge(video_data[video_data.item_id.notna()], how="left", on="item_id")
playback

,user_id,item_id,playtime,date,end_date,label,launch_date_len,launch_date,launch_type,launch_times,...,launch_seq_31,launch_seq_15,launch_seq_7,launch_times_31,launch_times_15,launch_times_7,duration,father_id,tag_list,cast
0,10057286,20628283,2208.612,145,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0,11,30038149,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...
1,10057286,20628283,332.351,150,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0,11,30038149,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...
2,10057286,24986210,74.370,161,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0,NaN,NaN,NaN,NaN
3,10057286,23148200,110.134,145,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0,NaN,NaN,NaN,NaN
4,10057286,23821440,1458.661,161,171,1,4,"[131, 145, 150, 161]","[0, 0, 0, 0]",4,...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",3,1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24425336,10450556,23026777,3209.858,144,163,0,2,"[142, 144]","[0, 0]",2,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",2,0,0,NaN,NaN,NaN,NaN
24425337,10073907,20928772,1.634,144,165,1,2,"[144, 151]","[1, 0]",2,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",2,1,0,6,30087116,50350272;50329038,NaN
24425338,10508542,20933156,4.070,144,173,0,2,"[144, 144]","[0, 1]",2,...,"[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,5,30004948,50260120;50294284;50319589;50297350;50323629;5...,40037610;40006409;40026897;40021404;40018177;4...
24425339,10380139,23439637,23.733,144,150,0,2,"[144, 144]","[0, 1]",2,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]","[0, 2, 0, 0, 0, 0, 0, 0]",1,1,1,11,30002513,50086696;50409449;50260120;50325616;50168087;5...,40040678;40033484;40013958;40004263;40036707;4...


In [45]:
# using target encoding
# Tutorial: https://www.kaggle.com/ryanholbrook/target-encoding
def target_encoding(name, df, m=1):
    df[name] = df[name].str.split(";")
    df = df.explode(name)
    overall = df["label"].mean()
    df = df.groupby(name).agg(
        freq=("label", "count"), 
        in_category=("label", np.mean)
    ).reset_index()
    df["weight"] = df["freq"] / (df["freq"] + m)
    df["score"] = df["weight"] * df["in_category"] + (1 - df["weight"]) * overall
    return df

In [46]:
# father_id target encoding
df = playback.loc[(playback.label >= 0) & (playback.father_id.notna()), ["father_id", "label"]]
father_id_score = target_encoding("father_id", df)
father_id_score

,father_id,freq,in_category,weight,score
0,30000000,3,6.666667,0.750000,6.010417
1,30000001,1,6.000000,0.500000,5.020834
2,30000002,5,3.000000,0.833333,3.173611
3,30000003,1,7.000000,0.500000,5.520834
4,30000005,4,4.250000,0.800000,4.208334
...,...,...,...,...,...
60411,30094642,4,0.250000,0.800000,1.008334
60412,30094643,41,2.146341,0.976190,2.191468
60413,30094644,1,4.000000,0.500000,4.020834
60414,30094645,20,5.000000,0.952381,4.954365


In [47]:
playback['tag_list'].head()

0    50444644;50260120;50057164;50323629;50107634;5...
1    50444644;50260120;50057164;50323629;50107634;5...
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: tag_list, dtype: object

In [48]:
#内存会爆
# tag_id target encoding
# df = playback.loc[(playback.label >= 0) & (playback.tag_list.notna()), ["tag_list", "label"]]
# tag_id_score = target_encoding("tag_list", df)
# tag_id_score.rename({"tag_list": "tag_id"}, axis=1, inplace=True)
# tag_id_score

In [49]:
# cast_id target encoding
df = playback.loc[(playback.label >= 0) & (playback.cast.notna()), ["cast", "label"]]
cast_id_score = target_encoding("cast", df)
cast_id_score.rename({"cast": "cast_id"}, axis=1, inplace=True)
cast_id_score

,cast_id,freq,in_category,weight,score
0,40000000,2,1.000000,0.666667,1.931144
1,40000001,1413,3.410474,0.999293,3.410745
2,40000004,105,3.447619,0.990566,3.450881
3,40000005,31,2.612903,0.968750,2.649795
4,40000009,13,3.538462,0.928571,3.556674
...,...,...,...,...,...
26843,40041231,244,4.524590,0.995918,4.521606
26844,40041235,8,4.250000,0.888889,4.199270
26845,40041236,107,3.570093,0.990741,3.572161
26846,40041237,71,3.943662,0.986111,3.941575


In [50]:
# group playback data for feature engineering
playback_grp = playback.groupby(["user_id", "end_date", "label"]).agg(
    playtime_list=("playtime", list),
    date_list=("date", list),
    duration_list=("duration", lambda x: ";".join(map(str, x))),
    father_id_list=("father_id", lambda x: ";".join(map(str, x))),
    tag_list=("tag_list", lambda x: ";".join(map(str, x))),
    cast_list=("cast", lambda x: ";".join(map(str, x)))
).reset_index()
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list
0,10000000,211,2,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...
1,10000002,128,1,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...
2,10000004,167,0,"[3.365, 1421.31, 855.577, 256.441, 1059.891, 1...","[154, 155, 157, 154, 154, 154, 154]",nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan
3,10000005,130,0,[1475.5829999999999],[124],nan,nan,nan,nan
4,10000007,152,0,"[4.888999999999999, 2.222, 181.75599999999997,...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan
...,...,...,...,...,...,...,...,...,...
439846,10599992,130,2,"[2482.9570000000003, 2791.7690000000002, 18.76...","[129, 129, 123, 129, 129, 123, 123, 129, 129, ...",nan;nan;nan;nan;nan;nan;nan;nan;nan;11;nan;nan...,nan;nan;nan;nan;nan;nan;nan;nan;nan;30089307;n...,nan;nan;nan;nan;nan;nan;nan;nan;nan;50252158;5...,nan;nan;nan;nan;nan;nan;nan;nan;nan;40001114;4...
439847,10599993,153,0,[3.7039999999999997],[153],6,30072863,50282198;50409449;50420012;50194452;50354161;5...,40023112;40013665;40034894;40018012;40018351;4...
439848,10599994,157,0,"[6.331, 502.05400000000003]","[151, 151]",nan;9,nan;30057840,nan;50401854;50294976;50096449;50207929;501570...,nan;nan
439849,10599995,180,1,"[536.882, 32.336999999999996, 110.372000000000...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...


In [51]:
# generate latest 32 days([end_date-31, end_date]) playtime sequence
# playtime_norm = 1/(1 + exp(3 - playtime/450)). when playtime=3600s it's preference score is almost equal to 1
def get_playtime_seq(row):
    seq_sort = sorted(zip(row.playtime_list, row.date_list), key=lambda x: x[1])
    seq_map = {k: sum(x[0] for x in g) for k, g in groupby(seq_sort, key=lambda x: x[1])}
    seq_norm = {k: 1/(1+np.exp(3-v/450)) for k, v in seq_map.items()}
    seq = [round(seq_norm.get(i, 0), 4) for i in range(row.end_date-31, row.end_date+1)]
    return seq
playback_grp["playtime_seq"] = playback_grp.apply(get_playtime_seq, axis=1)
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list,playtime_seq
0,10000000,211,2,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000002,128,1,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000004,167,0,"[3.365, 1421.31, 855.577, 256.441, 1059.891, 1...","[154, 155, 157, 154, 154, 154, 154]",nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10000005,130,0,[1475.5829999999999],[124],nan,nan,nan,nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000007,152,0,"[4.888999999999999, 2.222, 181.75599999999997,...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...
439846,10599992,130,2,"[2482.9570000000003, 2791.7690000000002, 18.76...","[129, 129, 123, 129, 129, 123, 123, 129, 129, ...",nan;nan;nan;nan;nan;nan;nan;nan;nan;11;nan;nan...,nan;nan;nan;nan;nan;nan;nan;nan;nan;30089307;n...,nan;nan;nan;nan;nan;nan;nan;nan;nan;50252158;5...,nan;nan;nan;nan;nan;nan;nan;nan;nan;40001114;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
439847,10599993,153,0,[3.7039999999999997],[153],6,30072863,50282198;50409449;50420012;50194452;50354161;5...,40023112;40013665;40034894;40018012;40018351;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
439848,10599994,157,0,"[6.331, 502.05400000000003]","[151, 151]",nan;9,nan;30057840,nan;50401854;50294976;50096449;50207929;501570...,nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
439849,10599995,180,1,"[536.882, 32.336999999999996, 110.372000000000...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...,"[0, 0, 0, 0, 0, 0, 0.9999, 0, 0, 0, 0, 0, 0, 0..."


In [52]:
drn_desc = video_data.loc[video_data.duration.notna(), "duration"].astype(int)
drn_desc.min(), drn_desc.max()

(1, 16)

In [53]:
# duration preference is a 16-dimentional prefer vector
# for a user, count the frequency of each duration
# prefer_score = freq / max(freq)
# if the user's duration_list is all null, then return null
# null duration_prefer will later be filled with zero vector
def get_duration_prefer(duration_list):
    drn_list = sorted(duration_list.split(";"))
    drn_map = {k: sum(1 for _ in g) for k, g in groupby(drn_list) if k != "nan"}
    if drn_map:
        max_ = max(drn_map.values())
        res = [round(drn_map.get(str(i), 0)/max_, 4) for i in range(1, 17)]
        return res
    else:
        return np.nan
playback_grp["duration_prefer"] = playback_grp.duration_list.apply(get_duration_prefer)

In [54]:
# add all target encoding scores into a dict
id_score = dict()
id_score.update({x[1]: x[5] for x in father_id_score.itertuples()})
#id_score.update({x[1]: x[5] for x in tag_id_score.itertuples()})
id_score.update({x[1]: x[5] for x in cast_id_score.itertuples()})

# check if features ids are duplicated
father_id_score.shape[0]+cast_id_score.shape[0] == len(id_score)

True

In [55]:
# for these 3 features: father_id_score, cast_score, tag_score,
# choose top 3 preferences
# e.g top_3_id = [(id, freq), (id, freq), (id, freq)]
# score = weight_avg(top_3_id), which values are id_score, weights are frequency
# if the id_list is all null, then return null
def get_id_score(id_list):
    x = sorted(id_list.split(";"))
    x_count = {k: sum(1 for _ in g) for k, g in groupby(x) if k != "nan"}
    if x_count:
        x_sort = sorted(x_count.items(), key=lambda k: -k[1])
        top_x = x_sort[:3]
        res = [(n, id_score.get(k, 0)) for k, n in top_x]
        res = sum(n*v for n, v in res) / sum(n for n, v in res)
        return res
    else:
        return np.nan

In [56]:
playback_grp["father_id_score"] = playback_grp.father_id_list.apply(get_id_score)

In [57]:
playback_grp["cast_id_score"] = playback_grp.cast_list.apply(get_id_score)

In [58]:
playback_grp["tag_score"] = playback_grp.tag_list.apply(get_id_score)
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score
0,10000000,211,2,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.357909,4.143713,0.0
1,10000002,128,1,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.985859,0.0
2,10000004,167,0,"[3.365, 1421.31, 855.577, 256.441, 1059.891, 1...","[154, 155, 157, 154, 154, 154, 154]",nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN
3,10000005,130,0,[1475.5829999999999],[124],nan,nan,nan,nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN
4,10000007,152,0,"[4.888999999999999, 2.222, 181.75599999999997,...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439846,10599992,130,2,"[2482.9570000000003, 2791.7690000000002, 18.76...","[129, 129, 123, 129, 129, 123, 123, 129, 129, ...",nan;nan;nan;nan;nan;nan;nan;nan;nan;11;nan;nan...,nan;nan;nan;nan;nan;nan;nan;nan;nan;30089307;n...,nan;nan;nan;nan;nan;nan;nan;nan;nan;50252158;5...,nan;nan;nan;nan;nan;nan;nan;nan;nan;40001114;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.513192,3.808690,0.0
439847,10599993,153,0,[3.7039999999999997],[153],6,30072863,50282198;50409449;50420012;50194452;50354161;5...,40023112;40013665;40034894;40018012;40018351;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",3.611392,3.614583,0.0
439848,10599994,157,0,"[6.331, 502.05400000000003]","[151, 151]",nan;9,nan;30057840,nan;50401854;50294976;50096449;50207929;501570...,nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",3.405360,NaN,0.0
439849,10599995,180,1,"[536.882, 32.336999999999996, 110.372000000000...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...,"[0, 0, 0, 0, 0, 0, 0.9999, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.4286, 0.8571,...",4.261891,3.676771,0.0


In [59]:
data = data.merge(
    playback_grp[["user_id", "end_date", "label", "playtime_seq", "duration_prefer", "father_id_score", "cast_id_score", "tag_score"]],
    on=["user_id", "end_date", "label"],
    how="left"
)
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,launch_times_15,launch_times_7,playtime_31,playtime_15,playtime_7,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,...,4,0,2587.170,0.000,0.000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.357909,4.143713,0.0
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000002,128,1,1,[128],[0],1,1,0,0.000000,...,1,1,19.925,19.925,19.925,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.985859,0.0
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.500000,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,...,4,0,5248.377,5248.377,0.000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634995,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,...,7,4,77650.347,40861.050,13927.542,"[0, 0.1903, 0, 0, 0, 0.0673, 0, 0, 0, 0, 0, 0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 1.0, 1.0, 0.5,...",3.851243,3.656183,0.0
634996,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,...,0,0,80.249,80.249,80.249,NaN,NaN,NaN,NaN,NaN
634997,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,...,0,0,55479.955,55479.955,55479.955,NaN,NaN,NaN,NaN,NaN
634998,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,...,0,0,42.292,0.000,0.000,"[0, 0, 0, 0, 0, 0.0505, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",NaN,NaN,0.0


In [60]:
data.head()

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,launch_times_15,launch_times_7,playtime_31,playtime_15,playtime_7,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.0,...,4,0,2587.170,0.000,0.000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.357909,4.143713,0.0
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000002,128,1,1,[128],[0],1,1,0,0.0,...,1,1,19.925,19.925,19.925,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.985859,0.0
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.0,...,4,0,5248.377,5248.377,0.000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN


# user portrait data process

In [61]:
portrait = pd.read_csv(input_dir + "user_portrait_data.csv", dtype={"territory_code": str})
portrait = pd.merge(data[["user_id", "label"]], portrait, how="left", on="user_id")
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code
0,10000000,2,2.0,7465,112369,1.0,4.0,0.0,1.0,864313
1,10000001,0,3.0,1021,15999,2.0,3.0,2.0,0.0,863100
2,10000002,1,1.0,3117,255978,2.0,4.0,2.0,1.0,861200
3,10000003,0,2.0,3799,54204,1.0,4.0,0.0,1.0,864304
4,10000004,0,2.0,11300,463992,2.0,3.0,0.0,0.0,862301
...,...,...,...,...,...,...,...,...,...,...
634996,10517398,-1,2.0,3571,114793,2.0,5.0,0.0,1.0,861200
634997,10110309,-1,1.0,2086,63989,2.0,3.0,0.0,1.0,863306
634998,10528272,-1,2.0,5880,55214,1.0,4.0,0.0,1.0,863213
634999,10395221,-1,2.0,7465,112369,1.0,4.0,0.0,1.0,863605


In [62]:
# for territory_code, using target encoding again
df = portrait.loc[(portrait.label >= 0) & (portrait.territory_code.notna()), ["territory_code", "label"]]
territory_score = target_encoding("territory_code", df)
territory_score

,territory_code,freq,in_category,weight,score
0,860000,1,0.000000,0.500000,0.674156
1,861100,12837,1.600919,0.999922,1.600900
2,861200,6621,1.503700,0.999849,1.503677
3,861301,4706,1.472588,0.999788,1.472562
4,861302,2879,1.347343,0.999653,1.347343
...,...,...,...,...,...
368,867107,14,2.000000,0.933333,1.956554
369,867108,38,1.289474,0.974359,1.290982
370,867109,11,0.454545,0.916667,0.529026
371,868100,126,1.015873,0.992126,1.018491


In [63]:
# add territory_code score into id_score
n1 = len(id_score)
id_score.update({x[1]: x[5] for x in territory_score.itertuples()})
n1 + territory_score.shape[0] == len(id_score)

True

In [64]:
# get territory score, retain null value
portrait["territory_score"] = portrait.territory_code.apply(lambda x: id_score.get(x, 0) if isinstance(x, str) else np.nan)
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,territory_score
0,10000000,2,2.0,7465,112369,1.0,4.0,0.0,1.0,864313,1.090736
1,10000001,0,3.0,1021,15999,2.0,3.0,2.0,0.0,863100,1.573680
2,10000002,1,1.0,3117,255978,2.0,4.0,2.0,1.0,861200,1.503677
3,10000003,0,2.0,3799,54204,1.0,4.0,0.0,1.0,864304,1.155789
4,10000004,0,2.0,11300,463992,2.0,3.0,0.0,0.0,862301,1.294725
...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,-1,2.0,3571,114793,2.0,5.0,0.0,1.0,861200,1.503677
634997,10110309,-1,1.0,2086,63989,2.0,3.0,0.0,1.0,863306,1.402056
634998,10528272,-1,2.0,5880,55214,1.0,4.0,0.0,1.0,863213,1.316857
634999,10395221,-1,2.0,7465,112369,1.0,4.0,0.0,1.0,863605,1.249546


In [65]:
# for multi values of device_ram and device_rom, choose the first one
portrait["device_ram"] = portrait.device_ram.apply(lambda x: float(x.split(";")[0]) if isinstance(x, str) else np.nan)
portrait["device_rom"] = portrait.device_rom.apply(lambda x: float(x.split(";")[0]) if isinstance(x, str) else np.nan)
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,territory_score
0,10000000,2,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,864313,1.090736
1,10000001,0,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,863100,1.573680
2,10000002,1,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,861200,1.503677
3,10000003,0,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,864304,1.155789
4,10000004,0,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,862301,1.294725
...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,-1,2.0,3571.0,114793.0,2.0,5.0,0.0,1.0,861200,1.503677
634997,10110309,-1,1.0,2086.0,63989.0,2.0,3.0,0.0,1.0,863306,1.402056
634998,10528272,-1,2.0,5880.0,55214.0,1.0,4.0,0.0,1.0,863213,1.316857
634999,10395221,-1,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,863605,1.249546


In [66]:
# add portrait features into data
data = data.merge(portrait.drop("territory_code", axis=1), how="left", on=["user_id", "label"])
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,...,4.143713,0.0,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.090736
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,...,NaN,NaN,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,1.573680
2,10000002,128,1,1,[128],[0],1,1,0,0.000000,...,2.985859,0.0,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,1.503677
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.500000,...,NaN,NaN,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,1.155789
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,...,NaN,NaN,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,1.294725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,...,3.656183,0.0,2.0,3571.0,114793.0,2.0,5.0,0.0,1.0,1.503677
634997,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,...,NaN,NaN,1.0,2086.0,63989.0,2.0,3.0,0.0,1.0,1.402056
634998,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,...,NaN,NaN,2.0,5880.0,55214.0,1.0,4.0,0.0,1.0,1.316857
634999,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,...,NaN,0.0,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.249546


# interaction data process

In [67]:
# only use interact_type preference
# use all interaction data to calculate interact_type preference
interact = pd.read_csv(input_dir + "user_interaction_data.csv")
interact.interact_type.min(), interact.interact_type.max()

(1, 11)

In [68]:
interact_grp = interact.groupby("user_id").agg(
    interact_type=("interact_type", list)
).reset_index()
interact_grp

,user_id,interact_type
0,10000069,"[9, 9]"
1,10000078,[10]
2,10000107,"[1, 1, 8, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,10000126,[1]
4,10000164,"[10, 10, 10]"
...,...,...
19941,10599761,"[10, 10, 9, 10, 7, 10, 7]"
19942,10599879,[7]
19943,10599887,[5]
19944,10599985,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."


In [69]:
# similar to duration preference, the interact_type preference could be a 11-dimentional vector
def get_interact_prefer(interact_type):
    x = sorted(interact_type)
    x_count = {k: sum(1 for _ in g) for k, g in groupby(x)}
    x_max = max(x_count.values())
    res = [round(x_count.get(i, 0)/x_max, 4) for i in range(1, 12)]
    return res
interact_grp["interact_prefer"] = interact_grp.interact_type.apply(get_interact_prefer)
interact_grp

,user_id,interact_type,interact_prefer
0,10000069,"[9, 9]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,10000078,[10],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000107,"[1, 1, 8, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 0.0, 0.0, 0.0556, 0.0, 0.0, 0.1667,..."
3,10000126,[1],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,10000164,"[10, 10, 10]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
19941,10599761,"[10, 10, 9, 10, 7, 10, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.25,..."
19942,10599879,[7],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
19943,10599887,[5],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
19944,10599985,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [70]:
data = data.merge(interact_grp[["user_id", "interact_prefer"]], on="user_id", how="left")
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.000000,...,0.0,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.090736,NaN
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.000000,...,NaN,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,1.573680,NaN
2,10000002,128,1,1,[128],[0],1,1,0,0.000000,...,0.0,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,1.503677,NaN
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.500000,...,NaN,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,1.155789,NaN
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.000000,...,NaN,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,1.294725,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36,36,0,0.000000,...,0.0,2.0,3571.0,114793.0,2.0,5.0,0.0,1.0,1.503677,NaN
634997,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",6,6,0,0.000000,...,NaN,1.0,2086.0,63989.0,2.0,3.0,0.0,1.0,1.402056,NaN
634998,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",22,20,2,0.090909,...,NaN,2.0,5880.0,55214.0,1.0,4.0,0.0,1.0,1.316857,NaN
634999,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",5,5,0,0.000000,...,0.0,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.249546,NaN


# feature normalization and save data

In [71]:
# filling null vector features with zero-vectors
data.fillna({
    "playtime_seq": str([0]*32),
    "duration_prefer": str([0]*16),
    "interact_prefer": str([0]*11)
}, inplace=True)

In [72]:
data.loc[data.label >= 0].to_csv(output_dir + "train_data_No_normaliza.txt", sep="\t", index=False)
data.loc[data.label < 0].to_csv(output_dir + "test_data_No_normaliza.txt", sep="\t", index=False)

In [73]:
data.head()

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",16,16,0,0.0,...,0.0,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.090736,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",3,3,0,0.0,...,NaN,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,1.573680,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,1,[128],[0],1,1,0,0.0,...,0.0,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,1.503677,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,199,0,2,"[144, 144]","[1, 0]",2,1,1,0.5,...,NaN,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,1.155789,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",4,4,0,0.0,...,NaN,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,1.294725,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [74]:
data.columns

Index(['user_id', 'end_date', 'label', 'launch_date_len', 'launch_date',
       'launch_type', 'launch_times', 'launch_type_0', 'launch_type_1',
       'launch_type_01rate', 'start_end_launch', 'launch_seq_31',
       'launch_seq_15', 'launch_seq_7', 'launch_times_31', 'launch_times_15',
       'launch_times_7', 'playtime_31', 'playtime_15', 'playtime_7',
       'playtime_seq', 'duration_prefer', 'father_id_score', 'cast_id_score',
       'tag_score', 'device_type', 'device_ram', 'device_rom', 'sex', 'age',
       'education', 'occupation_status', 'territory_score', 'interact_prefer'],
      dtype='object')

In [75]:
# the following features should be normalized
# method: x = (x - mean(x)) / std(x)
norm_cols = ["father_id_score", "cast_id_score", "tag_score", 
            "device_type", "device_ram", "device_rom", "sex",
            "age", "education", "occupation_status", "territory_score", 'launch_times', 'launch_type_0', 'launch_type_1', 'launch_type_01rate',
       'start_end_launch','launch_times_31', 'launch_times_15', 'launch_times_7', 'playtime_31',
       'playtime_15', 'playtime_7',]
for col in norm_cols:
    mean = data[col].mean()
    std = data[col].std()
    data[col] = (data[col] - mean) / std
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.849842,0.874528,-0.611652,-0.527202,...,NaN,0.194677,1.038058,-0.142070,-0.956632,0.827448,-0.545979,0.744456,-1.798070,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",-0.410684,-0.372292,-0.611652,-0.527202,...,NaN,2.427026,-2.051382,-1.254643,1.045333,-0.319627,2.052733,-1.343260,1.567918,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,1,[128],[0],-0.604611,-0.564111,-0.611652,-0.527202,...,NaN,-2.037672,-1.046499,1.515868,1.045333,0.827448,2.052733,0.744456,1.080016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,199,0,2,"[144, 144]","[1, 0]",-0.507648,-0.564111,1.117465,1.164124,...,NaN,0.194677,-0.719528,-0.813574,-0.956632,0.827448,-0.545979,0.744456,-1.344670,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",-0.313720,-0.276383,-0.611652,-0.527202,...,NaN,0.194677,2.876669,3.917349,1.045333,-0.319627,-0.545979,-1.343260,-0.376322,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.789112,2.792714,-0.611652,-0.527202,...,NaN,0.194677,-0.828838,-0.114085,1.045333,1.974522,-0.545979,0.744456,1.080016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634997,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",-0.119793,-0.084564,-0.611652,-0.527202,...,NaN,-2.037672,-1.540790,-0.700608,1.045333,-0.319627,-0.545979,0.744456,0.371749,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634998,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.431623,1.258165,2.846582,-0.219689,...,NaN,0.194677,0.278164,-0.801913,-0.956632,0.827448,-0.545979,0.744456,-0.222066,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634999,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",-0.216757,-0.180474,-0.611652,-0.527202,...,NaN,0.194677,1.038058,-0.142070,-0.956632,0.827448,-0.545979,0.744456,-0.691208,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [76]:
# filling null numeric features with 0
data.fillna(0, inplace=True)
data

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.849842,0.874528,-0.611652,-0.527202,...,0.0,0.194677,1.038058,-0.142070,-0.956632,0.827448,-0.545979,0.744456,-1.798070,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",-0.410684,-0.372292,-0.611652,-0.527202,...,0.0,2.427026,-2.051382,-1.254643,1.045333,-0.319627,2.052733,-1.343260,1.567918,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,1,[128],[0],-0.604611,-0.564111,-0.611652,-0.527202,...,0.0,-2.037672,-1.046499,1.515868,1.045333,0.827448,2.052733,0.744456,1.080016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,199,0,2,"[144, 144]","[1, 0]",-0.507648,-0.564111,1.117465,1.164124,...,0.0,0.194677,-0.719528,-0.813574,-0.956632,0.827448,-0.545979,0.744456,-1.344670,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",-0.313720,-0.276383,-0.611652,-0.527202,...,0.0,0.194677,2.876669,3.917349,1.045333,-0.319627,-0.545979,-1.343260,-0.376322,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634996,10517398,216,-1,36,"[132, 133, 134, 139, 144, 145, 147, 148, 149, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.789112,2.792714,-0.611652,-0.527202,...,0.0,0.194677,-0.828838,-0.114085,1.045333,1.974522,-0.545979,0.744456,1.080016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634997,10110309,172,-1,6,"[101, 111, 136, 142, 148, 154]","[0, 0, 0, 0, 0, 0]",-0.119793,-0.084564,-0.611652,-0.527202,...,0.0,-2.037672,-1.540790,-0.700608,1.045333,-0.319627,-0.545979,0.744456,0.371749,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634998,10528272,220,-1,22,"[140, 152, 153, 154, 155, 161, 164, 165, 166, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.431623,1.258165,2.846582,-0.219689,...,0.0,0.194677,0.278164,-0.801913,-0.956632,0.827448,-0.545979,0.744456,-0.222066,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
634999,10395221,195,-1,5,"[125, 144, 145, 146, 169]","[0, 0, 0, 0, 0]",-0.216757,-0.180474,-0.611652,-0.527202,...,0.0,0.194677,1.038058,-0.142070,-0.956632,0.827448,-0.545979,0.744456,-0.691208,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [77]:
# finally
data.loc[data.label >= 0].to_csv(output_dir + "train_data.txt", sep="\t", index=False)
data.loc[data.label < 0].to_csv(output_dir + "test_data.txt", sep="\t", index=False)

In [78]:
data.columns

Index(['user_id', 'end_date', 'label', 'launch_date_len', 'launch_date',
       'launch_type', 'launch_times', 'launch_type_0', 'launch_type_1',
       'launch_type_01rate', 'start_end_launch', 'launch_seq_31',
       'launch_seq_15', 'launch_seq_7', 'launch_times_31', 'launch_times_15',
       'launch_times_7', 'playtime_31', 'playtime_15', 'playtime_7',
       'playtime_seq', 'duration_prefer', 'father_id_score', 'cast_id_score',
       'tag_score', 'device_type', 'device_ram', 'device_rom', 'sex', 'age',
       'education', 'occupation_status', 'territory_score', 'interact_prefer'],
      dtype='object')